<a href="https://colab.research.google.com/github/DarkPrinceWarrior/Boston-Housing/blob/main/Boston_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from keras.datasets import boston_housing

(train_data, train_labels),(test_data, test_labels) = boston_housing.load_data()


In [21]:
# data preparation = normalization

# average value on the given axis (columns)
mean = train_data.mean(axis=0)

# substraction of mean from particular column data in dataset 
train_data -=mean
# standard deviation of columns
std = train_data.std(axis=0)

train_data/=std

test_data-=mean
test_data/=std



In [22]:
from keras import models 
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation="relu", input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64,activation="relu"))
  model.add(layers.Dense(1))
  model.compile(optimizer = "rmsprop",
    loss = "mse",
    metrics = ['mae'])
  return model

In [23]:
# K-fold cross-validation
import numpy as np

k = 4
num_epochs = 100
num_val_samples = len(train_data)//k
all_scores = []

for i in range(k):
  print("K model number: ",i)
  val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
  val_targets = train_labels[i*num_val_samples:(i+1)*num_val_samples]

  partial_train_data = np.concatenate([train_data[:i*num_val_samples], 
                              train_data[(i+1)*num_val_samples:]],axis=0)
  
  partial_train_targets = np.concatenate([train_labels[:i*num_val_samples], 
                              train_labels[(i+1)*num_val_samples:]],axis=0)
  
  model = build_model()
  model.fit(partial_train_data,partial_train_targets,
            epochs =num_epochs,
            batch_size = 1, verbose =0)
  
  val_mse, val_mae = model.evaluate(val_data,val_targets, verbose =0)
  all_scores.append(val_mae)

print(all_scores)
print(np.mean(all_scores))


K model number:  0
K model number:  1
K model number:  2
K model number:  3
[2.0027923583984375, 2.4924309253692627, 2.8038148880004883, 2.618802785873413]
2.4794602394104004


In [25]:
# final model learning
model = build_model()
model.fit(train_data,train_labels, 
          epochs=80,batch_size=16, verbose=0)
test_nse_score, test_mae_score = model.evaluate(test_data, test_labels)
test_mae_score

4/4 [==============================] - 0s 3ms/step - loss: 18.0367 - mae: 2.7341


2.73407244682312